In [19]:
import csv
import datetime
import random
from operator import sub
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn import preprocessing, ensemble

mapping_dict = {'ind_empleado'  : {-99:0, 'N':1, 'B':2, 'F':3, 'A':4, 'S':5},
'sexo'          : {'V':0, 'H':1, -99:2},
'ind_nuevo'     : {'0':0, '1':1, -99:1},
'indrel'        : {'1':0, '99':1, -99:1},
'indrel_1mes'   : {-99:0, '1.0':1, '1':1, '2.0':2, '2':2, '3.0':3, '3':3, '4.0':4, '4':4, 'P':5},
'tiprel_1mes'   : {-99:0, 'I':1, 'A':2, 'P':3, 'R':4, 'N':5},
'indresi'       : {-99:0, 'S':1, 'N':2},
'indext'        : {-99:0, 'S':1, 'N':2},
#'conyuemp'      : {-99:0, 'S':1, 'N':2},
'indfall'       : {-99:0, 'S':1, 'N':2},
#'tipodom'       : {-99:0, '1':1},
'ind_actividad_cliente' : {'0':0, '1':1, -99:2},
'segmento'      : {'02 - PARTICULARES':0, '03 - UNIVERSITARIO':1, '01 - TOP':2, -99:3},
'pais_residencia' : {'LV': 102, 'BE': 12, 'BG': 50, 'BA': 61, 'BM': 117, 'BO': 62, 'JP': 82, 'JM': 116, 'BR': 17, 'BY': 64, 'BZ': 113, 'RU': 43, 'RS': 89, 'RO': 41, 'GW': 99, 'GT': 44, 'GR': 39, 'GQ': 73, 'GE': 78, 'GB': 9, 'GA': 45, 'GN': 98, 'GM': 110, 'GI': 96, 'GH': 88, 'OM': 100, 'HR': 67, 'HU': 106, 'HK': 34, 'HN': 22, 'AD': 35, 'PR': 40, 'PT': 26, 'PY': 51, 'PA': 60, 'PE': 20, 'PK': 84, 'PH': 91, 'PL': 30, 'EE': 52, 'EG': 74, 'ZA': 75, 'EC': 19, 'AL': 25, 'VN': 90, 'ET': 54, 'ZW': 114, 'ES': 0, 'MD': 68, 'UY': 77, 'MM': 94, 'ML': 104, 'US': 15, 'MT': 118, 'MR': 48, 'UA': 49, 'MX': 16, 'IL': 42, 'FR': 8, 'MA': 38, 'FI': 23, 'NI': 33, 'NL': 7, 'NO': 46, 'NG': 83, 'NZ': 93, 'CI': 57, 'CH': 3, 'CO': 21, 'CN': 28, 'CM': 55, 'CL': 4, 'CA': 2, 'CG': 101, 'CF': 109, 'CD': 112, 'CZ': 36, 'CR': 32, 'CU': 72, 'KE': 65, 'KH': 95, 'SV': 53, 'SK': 69, 'KR': 87, 'KW': 92, 'SN': 47, 'SL': 97, 'KZ': 111, 'SA': 56, 'SG': 66, 'SE': 24, 'DO': 11, 'DJ': 115, 'DK': 76, 'DE': 10, 'DZ': 80, 'MK': 105, -99: 1, 'LB': 81, 'TW': 29, 'TR': 70, 'TN': 85, 'LT': 103, 'LU': 59, 'TH': 79, 'TG': 86, 'LY': 108, 'AE': 37, 'VE': 14, 'IS': 107, 'IT': 18, 'AO': 71, 'AR': 13, 'AU': 63, 'AT': 6, 'IN': 31, 'IE': 5, 'QA': 58, 'MZ': 27},
'canal_entrada' : {'013': 49, 'KHP': 160, 'KHQ': 157, 'KHR': 161, 'KHS': 162, 'KHK': 10, 'KHL': 0, 'KHM': 12, 'KHN': 21, 'KHO': 13, 'KHA': 22, 'KHC': 9, 'KHD': 2, 'KHE': 1, 'KHF': 19, '025': 159, 'KAC': 57, 'KAB': 28, 'KAA': 39, 'KAG': 26, 'KAF': 23, 'KAE': 30, 'KAD': 16, 'KAK': 51, 'KAJ': 41, 'KAI': 35, 'KAH': 31, 'KAO': 94, 'KAN': 110, 'KAM': 107, 'KAL': 74, 'KAS': 70, 'KAR': 32, 'KAQ': 37, 'KAP': 46, 'KAW': 76, 'KAV': 139, 'KAU': 142, 'KAT': 5, 'KAZ': 7, 'KAY': 54, 'KBJ': 133, 'KBH': 90, 'KBN': 122, 'KBO': 64, 'KBL': 88, 'KBM': 135, 'KBB': 131, 'KBF': 102, 'KBG': 17, 'KBD': 109, 'KBE': 119, 'KBZ': 67, 'KBX': 116, 'KBY': 111, 'KBR': 101, 'KBS': 118, 'KBP': 121, 'KBQ': 62, 'KBV': 100, 'KBW': 114, 'KBU': 55, 'KCE': 86, 'KCD': 85, 'KCG': 59, 'KCF': 105, 'KCA': 73, 'KCC': 29, 'KCB': 78, 'KCM': 82, 'KCL': 53, 'KCO': 104, 'KCN': 81, 'KCI': 65, 'KCH': 84, 'KCK': 52, 'KCJ': 156, 'KCU': 115, 'KCT': 112, 'KCV': 106, 'KCQ': 154, 'KCP': 129, 'KCS': 77, 'KCR': 153, 'KCX': 120, 'RED': 8, 'KDL': 158, 'KDM': 130, 'KDN': 151, 'KDO': 60, 'KDH': 14, 'KDI': 150, 'KDD': 113, 'KDE': 47, 'KDF': 127, 'KDG': 126, 'KDA': 63, 'KDB': 117, 'KDC': 75, 'KDX': 69, 'KDY': 61, 'KDZ': 99, 'KDT': 58, 'KDU': 79, 'KDV': 91, 'KDW': 132, 'KDP': 103, 'KDQ': 80, 'KDR': 56, 'KDS': 124, 'K00': 50, 'KEO': 96, 'KEN': 137, 'KEM': 155, 'KEL': 125, 'KEK': 145, 'KEJ': 95, 'KEI': 97, 'KEH': 15, 'KEG': 136, 'KEF': 128, 'KEE': 152, 'KED': 143, 'KEC': 66, 'KEB': 123, 'KEA': 89, 'KEZ': 108, 'KEY': 93, 'KEW': 98, 'KEV': 87, 'KEU': 72, 'KES': 68, 'KEQ': 138, -99: 6, 'KFV': 48, 'KFT': 92, 'KFU': 36, 'KFR': 144, 'KFS': 38, 'KFP': 40, 'KFF': 45, 'KFG': 27, 'KFD': 25, 'KFE': 148, 'KFB': 146, 'KFC': 4, 'KFA': 3, 'KFN': 42, 'KFL': 34, 'KFM': 141, 'KFJ': 33, 'KFK': 20, 'KFH': 140, 'KFI': 134, '007': 71, '004': 83, 'KGU': 149, 'KGW': 147, 'KGV': 43, 'KGY': 44, 'KGX': 24, 'KGC': 18, 'KGN': 11}
}
cat_cols = list(mapping_dict.keys())
target_cols = ['ind_ahor_fin_ult1','ind_aval_fin_ult1','ind_cco_fin_ult1','ind_cder_fin_ult1','ind_cno_fin_ult1','ind_ctju_fin_ult1','ind_ctma_fin_ult1','ind_ctop_fin_ult1','ind_ctpp_fin_ult1','ind_deco_fin_ult1','ind_deme_fin_ult1','ind_dela_fin_ult1','ind_ecue_fin_ult1','ind_fond_fin_ult1','ind_hip_fin_ult1','ind_plan_fin_ult1','ind_pres_fin_ult1','ind_reca_fin_ult1','ind_tjcr_fin_ult1','ind_valo_fin_ult1','ind_viv_fin_ult1','ind_nomina_ult1','ind_nom_pens_ult1','ind_recibo_ult1']
target_cols = target_cols[2:]

def getTarget(row):
	tlist = []
	for col in target_cols:
		if row[col].strip() in ['', 'NA']:
			target = 0
		else:
			target = int(float(row[col]))
		tlist.append(target)
	return tlist

def getTarget(row):
	tlist = []
	for col in target_cols:
		if row[col].strip() in ['', 'NA']:
			target = 0
		else:
			target = int(float(row[col]))
		tlist.append(target)
	return tlist

def getIndex(row, col):
	val = row[col].strip()
	if val not in ['','NA']:
		ind = mapping_dict[col][val]
	else:
		ind = mapping_dict[col][-99]
	return ind

def getAge(row):
	mean_age = 40.
	min_age = 18.
	max_age = 100.
	range_age = max_age - min_age
	age = row['age'].strip()
	if age == 'NA' or age == '':
		age = mean_age
	else:
		age = float(age)
		if age < min_age:
			age = min_age
		elif age > max_age:
			age = max_age
	return age

def getCustSeniority(row):
	min_value = 0.
	max_value = 256.
	range_value = max_value - min_value
	missing_value = 0.
	cust_seniority = row['antiguedad'].strip()
	if cust_seniority == 'NA' or cust_seniority == '':
		cust_seniority = missing_value
	else:
		cust_seniority = float(cust_seniority)
		if cust_seniority < min_value:
			cust_seniority = min_value
		elif cust_seniority > max_value:
			cust_seniority = max_value
	return cust_seniority

def getRent(row):
    min_value = 0.
    max_value = 1500000.
    range_value = max_value - min_value
    renta_dict = {'ALBACETE': 76895,  'ALICANTE': 60562,  'ALMERIA': 77815,  'ASTURIAS': 83995,  'AVILA': 78525,  'BADAJOZ': 60155,  'BALEARS, ILLES': 114223,  'BARCELONA': 135149,  'BURGOS': 87410, 'NAVARRA' : 101850,
    'CACERES': 78691,  'CADIZ': 75397,  'CANTABRIA': 87142,  'CASTELLON': 70359,  'CEUTA': 333283, 'CIUDAD REAL': 61962,  'CORDOBA': 63260, 'CORU脩A, A' : 103567,  'CUENCA': 70751,  'GIRONA': 100208,  'GRANADA': 80489,
    'GUADALAJARA': 100635,  'HUELVA': 75534,  'HUESCA': 80324,  'JAEN': 67016,  'LEON': 76339,  'LERIDA': 59191,  'LUGO': 68219,  'MADRID': 141381,  'MALAGA': 89534,  'MELILLA': 116469, 'GIPUZKOA': 101850,
    'MURCIA': 68713,  'OURENSE': 78776,  'PALENCIA': 90843,  'PALMAS, LAS': 78168,  'PONTEVEDRA': 94328,  'RIOJA, LA': 91545,  'SALAMANCA': 88738,  'SANTA CRUZ DE TENERIFE': 83383, 'ALAVA': 101850, 'BIZKAIA' : 101850,
    'SEGOVIA': 81287,  'SEVILLA': 94814,  'SORIA': 71615,  'TARRAGONA': 81330,  'TERUEL': 64053,  'TOLEDO': 65242,  'UNKNOWN': 103689,  'VALENCIA': 73463,  'VALLADOLID': 92032,  'ZAMORA': 73727,  'ZARAGOZA': 98827}

    #missing_value = 101850.
    rent = row['renta'].strip()
    if rent == 'NA' or rent == '':
        if row['nomprov'] == 'NA' or row['nomprov'] == '':
            rent = float(renta_dict['UNKNOWN'])
        else:
            try:
                rent = float(renta_dict[row['nomprov']])
            except:
                rent = float(renta_dict['UNKNOWN'])      
    else:
        rent = float(rent)
        if rent < min_value:
            rent = min_value
        elif rent > max_value:
            rent = max_value

    return rent

def getMarriageIndex(row, age, sex, income):
    marriage_age = 28
    modifier = 0
    if sex == 'V':
        modifier += -2
    if income <= 101850:
        modifier += -1
    
    marriage_age_mod = marriage_age + modifier
    
    if age <= marriage_age_mod:
        return 0
    else:
        return 1

def getMonth(row):
    return int(row['fecha_dato'].split('-')[1])

def getUserid(row):
    return row['ncodpers']

def getjoinMonth(row):
    if row['fecha_alta'].strip() == 'NA' or row['fecha_alta'].strip() == '':
        return int(random.choice([1,2,3,4,5,6,7,8,9,10,11,12]))
    else:
        return int(row['fecha_alta'].split('-')[1])


def processData(in_file_name, cust_dict):
	x_vars_list = []
	y_vars_list = []
	for row in csv.DictReader(in_file_name):
		# use only the four months as specified by breakfastpirate #
		if row['fecha_dato'] not in ['2015-05-28', '2015-06-28','2016-06-28','2016-05-28']:
			continue

		cust_id = int(row['ncodpers'])
		if row['fecha_dato'] in ['2015-05-28','2016-05-28']:
			target_list = getTarget(row)
			cust_dict[cust_id] =  target_list[:]
			continue

		x_vars = []
		userid = getUserid(row)
		x_vars.append(userid)
		for col in cat_cols:
			x_vars.append( getIndex(row, col) )
		sex = getIndex(row, 'sexo')
		age = getAge(row)
		x_vars.append( age )
		x_vars.append( getCustSeniority(row) )
		income = getRent(row) 
		x_vars.append( income )
		x_vars.append( getMarriageIndex(row, age, sex, income) )

		if row['fecha_dato'] == '2016-0  6-28':
			prev_target_list = cust_dict.get(cust_id, [0]*22)
			x_vars_list.append(x_vars + prev_target_list)
		elif row['fecha_dato'] == '2015-06-28':
			prev_target_list = cust_dict.get(cust_id, [0]*22)
			target_list = getTarget(row)
			new_products = [max(x1 - x2,0) for (x1, x2) in zip(target_list, prev_target_list)]
			if sum(new_products) > 0:
				for ind, prod in enumerate(new_products):
					if prod>0:
						assert len(prev_target_list) == 22
						x_vars_list.append(x_vars+prev_target_list)
						y_vars_list.append(ind)

	return x_vars_list, y_vars_list, cust_dict


In [23]:
data_path = ""
train_file =  open(data_path + "train_ver2.csv/train_ver2.csv")
print('Starting file processing')
x_vars_list, y_vars_list, cust_dict = processData(train_file, {})

print('Finished file processing')
train_X = np.array(x_vars_list)
train_y = np.array(y_vars_list)

Starting file processing
Finished file processing


In [24]:
print(np.unique(train_y))
del x_vars_list, y_vars_list
train_file.close()
print(train_X.shape, train_y.shape)

#test_file = open(data_path + "test_ver2.csv/test_ver2.csv")
#x_vars_list, y_vars_list, cust_dict = processData(test_file, cust_dict)
#test_X = np.array(x_vars_list)
#del x_vars_list
#test_file.close()
#print(test_X.shape)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]
(45679, 40) (45679,)


In [25]:
train_X_new = train_X[:,:-22]
#test_X_new = test_X[:,:-22]
train_X_new = pd.DataFrame(train_X_new,columns=['user_id','ind_empleado','sex','ind_nuevo','indrel','indrel_1mes','tiprel_1mes','indresi','indext','indfall','ind_actividad_cliente','segmento','pais_residencia','canal_entrada','age','cust_seniority','income','marriage'])
#test_X_new = pd.DataFrame(test_X_new,columns=['user_id','ind_empleado','sex','ind_nuevo','indrel','indrel_1mes','tiprel_1mes','indresi','indext','indfall','ind_actividad_cliente','segmento','pais_residencia','canal_entrada','age','cust_seniority','income','marriage'])

data = train_X_new
#test = test_X_new
y = train_y
y = pd.Series(y,name='item_id')
y = y+2

new = pd.concat([data,y],1)
#new.rename(columns={'0':'product'}, inplace = True)
y = np.ones([45679,1])
new = pd.concat([new,pd.DataFrame(y,columns=['target'])],1)

In [26]:
data = data.drop_duplicates()

test_li = []
k = data.user_id
for i in k:
    for j in range(2,24):
        tmp = np.array([i,j,0])
        test_li.append(tmp)

In [27]:
## single to multi
test_li = pd.DataFrame(np.array(test_li))
test_li.columns = ['user_id','item_id','target']

a = pd.merge(data,test_li,on=['user_id'],how='inner')
b = new[['user_id','item_id','target']]
a.user_id = a.user_id.astype(np.int32)
b.user_id = b.user_id.astype(np.int32)
a.item_id = a.item_id.astype(np.int32)
b.item_id = b.item_id.astype(np.int32)

final = pd.merge(a, b, on=['user_id','item_id'],how='left')
final = final.fillna(0)
final = final.drop(['target_x'],1)
final.rename(columns={'target_y':'target'},inplace=True)
train = final

C:\Users\64161\Anaconda3\lib\site-packages\pandas\core\generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [18]:
# test = test_X_new
# test_li1 = []
# k = test.user_id
# for i in k:
#     for j in range(2,24):
#         tmp = np.array([i,j])
#         test_li1.append(tmp)
        
# test_1 = pd.DataFrame(np.array(test_li1))
# test_1.columns = ['user_id','item_id']
# test_1.userid = test_1.user_id.astype(np.int32)
# test_1 = pd.merge(test,test_1,on='user_id')
# test = pd.DataFrame(test_1)

In [28]:
item = train.item_id
train.insert(0,'item',item)
order = ['ind_empleado', 'sex', 'ind_nuevo', 'indrel', 'indrel_1mes',
       'tiprel_1mes', 'indresi', 'indext', 'indfall', 'ind_actividad_cliente',
       'segmento', 'pais_residencia', 'canal_entrada', 'age', 'cust_seniority',
       'income', 'marriage','item', 'user_id', 'item_id', 'target']
train = train[order]
train.to_csv('train.csv',index=None)
#one = pd.Series(np.ones([test.shape[0],1]).ravel())
#test.insert(0,'target',one)
#item = test.item_id
#test.insert(0,'item',item)

#order = ['ind_empleado', 'sex', 'ind_nuevo', 'indrel', 'indrel_1mes',
#       'tiprel_1mes', 'indresi', 'indext', 'indfall', 'ind_actividad_cliente',
#       'segmento', 'pais_residencia', 'canal_entrada', 'age', 'cust_seniority',
#       'income', 'marriage','item', 'user_id', 'item_id','target']
#test = test[order]


#test.to_csv('test.csv',index=None)

In [29]:
import pandas as pd
import numpy as np
data = pd.read_csv('train.csv')
user_id = data.user_id.unique()
item_id = data.item_id.unique()

In [30]:
pnr = data[data.target==1].shape[0]/data.shape[0]*0.5

In [31]:
pnr

0.028280007924518957

## exposure filling

In [89]:
ratio = 1/(1/(data[data.target==1].item_id.value_counts()/data.item_id.value_counts())-1)*2

In [91]:
ratio = dict(zip(ratio.index,ratio.values))

In [92]:
ratio

{2: 0.6940153377609803,
 3: 0.0004904498515026839,
 4: 0.11122613296526339,
 5: 0.003000954849270223,
 6: 0.01919639173840103,
 7: 0.012168384126288095,
 8: 0.008425429478061058,
 9: 0.027784682519954706,
 10: 0.001799492870191128,
 11: 0.060912280701754376,
 12: 0.06869347158434533,
 13: 0.013492759982448444,
 14: 0.00021794801939737372,
 15: 0.0011447572841996238,
 16: 0.0004359435453108822,
 17: 0.17424780857616676,
 18: 0.2976060084493819,
 19: 0.008700172361905283,
 20: 0.00016345656141880294,
 21: 0.3271736029668136,
 22: 0.577858923492855,
 23: 0.6621704920410458}

In [93]:
from sklearn.model_selection import train_test_split

In [94]:
neg = data[data.target==0]
pos = data[data.target==1]

In [95]:
att = []
for i in range(len(ratio)):
    att1 = neg[neg.item_id==2+i]
    att1, _ = train_test_split(att1,train_size=ratio[2+i])
    att.append(att1)

In [96]:
new_neg = pd.concat(att,0)

In [97]:
data = pd.concat([new_neg,pos],0)

In [98]:
data.to_csv('train_new_2.csv',index=None)

## mf filling

In [44]:
data[data.target==1].shape

(45679, 21)

In [32]:
def data_index(train_x):
    df = pd.DataFrame(train_x)
    feat_dict = {}
    ui_shape = []
    tc = 0
    for col in range(train_x.shape[1]):
        us = df[col].unique()
        ui_shape.append(us.shape[0])
        feat_dict[col] = dict(zip(us, range(tc, len(us)+tc)))
        tc += len(us)
        feat_dim = tc

    return feat_dict, feat_dim, ui_shape

def data_index_get(train_x, feat_dict):

    dfi = pd.DataFrame(train_x)
    for col in dfi.columns:
        dfi[col] = dfi[col].map(feat_dict[col])
    Xi = dfi.values.tolist()
    Xi = np.array(Xi)

    return Xi

def predict(Xi,filled_matrix):
    pred2 = []
    for i in range(Xi.shape[0]):
        pred2.append(filled_matrix[int(Xi[i,0]),int(Xi[i,1])])
    pred2 = np.ravel(np.array(pred2))
    final_pred = pred2
    return final_pred

In [33]:
Xi= data.iloc[:,-3:]
one_s = Xi[Xi.target==1].iloc[:,:-1]
one_s = one_s.values
Xi= data.iloc[:,-3:-1].values
feat_dict, feat_dim, ui_shape = data_index(one_s)
Xi_ = data_index_get(one_s,feat_dict)
Xi_[:,-1] = Xi_[:,-1]-ui_shape[0]
Xi__ = data_index_get(Xi,feat_dict)
Xi__[:,-1] = Xi__[:,-1]-ui_shape[0]

In [34]:
train_index = Xi_
matrix =np.zeros(shape=[ui_shape[0],ui_shape[1]])
for i in range(train_index.shape[0]):
    matrix[int(train_index[i,0]),int(train_index[i,1])] = 1
matrix[matrix==0] = np.nan

print('missing value counts:',np.isnan(matrix).sum())
 
import sys
sys.path.append('fancyimpute_/')
#from fancyimpute_.soft_impute import SoftImpute
from fancyimpute_.fancyimpute_.soft_impute import SoftImpute
X_filled_softimpute = SoftImpute(max_iters=1000,max_rank=10).fit_transform(matrix)

missing value counts: 761941
[[1. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[SoftImpute] Max Singular Value of X_init = 111.099224
[SoftImpute] Iter 1: observed MAE=0.056491 rank=10
[SoftImpute] Iter 2: observed MAE=0.055919 rank=10
[SoftImpute] Iter 3: observed MAE=0.053400 rank=10
[SoftImpute] Iter 4: observed MAE=0.051477 rank=10
[SoftImpute] Iter 5: observed MAE=0.050021 rank=10
[SoftImpute] Iter 6: observed MAE=0.048637 rank=10
[SoftImpute] Iter 7: observed MAE=0.047442 rank=10
[SoftImpute] Iter 8: observed MAE=0.046565 rank=10
[SoftImpute] Iter 9: observed MAE=0.045951 rank=10
[SoftImpute] Iter 10: observed MAE=0.045445 rank=10
[SoftImpute] Iter 11: observed MAE=0.044828 rank=10
[SoftImpute] Iter 12: observed MAE=0.043887 rank=10
[SoftImpute] Iter 13: observed MAE=0.042596 rank=10
[SoftImpute] Iter 14: observed MAE=0.041123 rank=10
[SoftImpute] Iter 15: observed MAE=0.0396

[SoftImpute] Iter 156: observed MAE=0.032985 rank=10
[SoftImpute] Iter 157: observed MAE=0.032982 rank=10
[SoftImpute] Iter 158: observed MAE=0.032979 rank=10
[SoftImpute] Iter 159: observed MAE=0.032976 rank=10
[SoftImpute] Iter 160: observed MAE=0.032973 rank=10
[SoftImpute] Iter 161: observed MAE=0.032970 rank=10
[SoftImpute] Iter 162: observed MAE=0.032968 rank=10
[SoftImpute] Iter 163: observed MAE=0.032965 rank=10
[SoftImpute] Iter 164: observed MAE=0.032962 rank=10
[SoftImpute] Iter 165: observed MAE=0.032959 rank=10
[SoftImpute] Iter 166: observed MAE=0.032957 rank=10
[SoftImpute] Iter 167: observed MAE=0.032954 rank=10
[SoftImpute] Iter 168: observed MAE=0.032952 rank=10
[SoftImpute] Iter 169: observed MAE=0.032949 rank=10
[SoftImpute] Iter 170: observed MAE=0.032947 rank=10
[SoftImpute] Iter 171: observed MAE=0.032944 rank=10
[SoftImpute] Iter 172: observed MAE=0.032942 rank=10
[SoftImpute] Iter 173: observed MAE=0.032940 rank=10
[SoftImpute] Iter 174: observed MAE=0.032937 r

[SoftImpute] Iter 313: observed MAE=0.032764 rank=10
[SoftImpute] Iter 314: observed MAE=0.032763 rank=10
[SoftImpute] Iter 315: observed MAE=0.032762 rank=10
[SoftImpute] Iter 316: observed MAE=0.032761 rank=10
[SoftImpute] Iter 317: observed MAE=0.032760 rank=10
[SoftImpute] Iter 318: observed MAE=0.032759 rank=10
[SoftImpute] Iter 319: observed MAE=0.032758 rank=10
[SoftImpute] Iter 320: observed MAE=0.032757 rank=10
[SoftImpute] Iter 321: observed MAE=0.032756 rank=10
[SoftImpute] Iter 322: observed MAE=0.032755 rank=10
[SoftImpute] Iter 323: observed MAE=0.032754 rank=10
[SoftImpute] Iter 324: observed MAE=0.032753 rank=10
[SoftImpute] Iter 325: observed MAE=0.032752 rank=10
[SoftImpute] Iter 326: observed MAE=0.032751 rank=10
[SoftImpute] Iter 327: observed MAE=0.032749 rank=10
[SoftImpute] Iter 328: observed MAE=0.032748 rank=10
[SoftImpute] Iter 329: observed MAE=0.032747 rank=10
[SoftImpute] Iter 330: observed MAE=0.032746 rank=10
[SoftImpute] Iter 331: observed MAE=0.032745 r

[SoftImpute] Iter 469: observed MAE=0.032626 rank=10
[SoftImpute] Iter 470: observed MAE=0.032626 rank=10
[SoftImpute] Iter 471: observed MAE=0.032626 rank=10
[SoftImpute] Iter 472: observed MAE=0.032626 rank=10
[SoftImpute] Iter 473: observed MAE=0.032625 rank=10
[SoftImpute] Iter 474: observed MAE=0.032625 rank=10
[SoftImpute] Iter 475: observed MAE=0.032625 rank=10
[SoftImpute] Iter 476: observed MAE=0.032625 rank=10
[SoftImpute] Iter 477: observed MAE=0.032625 rank=10
[SoftImpute] Iter 478: observed MAE=0.032625 rank=10
[SoftImpute] Iter 479: observed MAE=0.032625 rank=10
[SoftImpute] Iter 480: observed MAE=0.032625 rank=10
[SoftImpute] Iter 481: observed MAE=0.032625 rank=10
[SoftImpute] Iter 482: observed MAE=0.032624 rank=10
[SoftImpute] Iter 483: observed MAE=0.032624 rank=10
[SoftImpute] Iter 484: observed MAE=0.032624 rank=10
[SoftImpute] Iter 485: observed MAE=0.032624 rank=10
[SoftImpute] Iter 486: observed MAE=0.032624 rank=10
[SoftImpute] Iter 487: observed MAE=0.032624 r

In [19]:
X_filled_softimpute

array([[ 1.00000000e+00,  1.00000000e+00,  1.86005440e-01, ...,
         5.04132885e-02,  1.48380720e-02,  1.89796263e-04],
       [ 1.41049644e-01,  2.19296877e-01,  1.00000000e+00, ...,
         5.40507078e-02,  4.62942711e-04, -3.27706154e-06],
       [ 1.50929056e-01,  2.11138473e-01,  1.85834978e-01, ...,
         7.63277391e-02, -3.61360109e-03,  4.62221478e-04],
       ...,
       [ 2.04846837e-01,  1.00000000e+00,  1.33010451e-01, ...,
         1.02639353e-02, -1.38193338e-03,  3.28691489e-06],
       [ 1.50929056e-01,  2.11138473e-01,  1.85834978e-01, ...,
         7.63277391e-02, -3.61360109e-03,  4.62221478e-04],
       [ 1.17786838e-01,  1.95583120e-01,  2.85903110e-01, ...,
         9.21439557e-02,  1.02162386e-04, -9.38281446e-04]])

In [48]:
X_filled_softimpute[X_filled_softimpute>0.5].shape

(62216,)

In [27]:
X_filled_softimpute[(X_filled_softimpute>0.05)&(X_filled_softimpute<=0.5)].shape

(530413,)

In [36]:
X_filled_softimpute[X_filled_softimpute<=pnr].shape

(343087,)

In [6]:
X_filled_softimpute[X_filled_softimpute>0.5].shape

(263570,)

In [8]:
X_filled_softimpute[X_filled_softimpute<10000].shape

(807620,)

In [37]:
X_filled_softimpute[X_filled_softimpute==1] =1
X_filled_softimpute[(X_filled_softimpute!=1)&(X_filled_softimpute>pnr)] = 2
X_filled_softimpute[X_filled_softimpute<=pnr] =0

res = predict(Xi__,X_filled_softimpute)
data.target = res
data = data[data.target!=2]
#data.to_csv('train_new_2.csv',index=None)

In [38]:
data.target.value_counts()

0.0    343087
1.0     45679
Name: target, dtype: int64

In [39]:
mapping_dict = {
'sexo'          : {'V':0, 'H':1, -99:2},
'ind_nuevo'     : {'0':0, '1':1, -99:1},
'indrel'        : {'1':0, '99':1, -99:1},

'indresi'       : {-99:0, 'S':1, 'N':2},
'indext'        : {-99:0, 'S':1, 'N':2}}

ind_em = {1:'not employee',2:'ex employed',3:'F filial',4:'active',0:'others'}
data_ind = []
for i in data.ind_empleado:
    data_ind.append(ind_em[i])
data.ind_empleado = data_ind

ind_mes = {1:'Primary customer',2:'co-owner',3:'former primary',0:'others'}
data_indm = []
for i in data.indrel_1mes:
    data_indm.append(ind_mes[i])
data.indrel_1mes = data_indm

pais_residencia = {'LV': 102, 'BE': 12, 'BG': 50, 'BA': 61, 'BM': 117, 'BO': 62, 'JP': 82, 'JM': 116, 'BR': 17, 'BY': 64, 'BZ': 113, 'RU': 43, 'RS': 89, 'RO': 41, 'GW': 99, 'GT': 44, 'GR': 39, 'GQ': 73, 'GE': 78, 'GB': 9, 'GA': 45, 'GN': 98, 'GM': 110, 'GI': 96, 'GH': 88, 'OM': 100, 'HR': 67, 'HU': 106, 'HK': 34, 'HN': 22, 'AD': 35, 'PR': 40, 'PT': 26, 'PY': 51, 'PA': 60, 'PE': 20, 'PK': 84, 'PH': 91, 'PL': 30, 'EE': 52, 'EG': 74, 'ZA': 75, 'EC': 19, 'AL': 25, 'VN': 90, 'ET': 54, 'ZW': 114, 'ES': 0, 'MD': 68, 'UY': 77, 'MM': 94, 'ML': 104, 'US': 15, 'MT': 118, 'MR': 48, 'UA': 49, 'MX': 16, 'IL': 42, 'FR': 8, 'MA': 38, 'FI': 23, 'NI': 33, 'NL': 7, 'NO': 46, 'NG': 83, 'NZ': 93, 'CI': 57, 'CH': 3, 'CO': 21, 'CN': 28, 'CM': 55, 'CL': 4, 'CA': 2, 'CG': 101, 'CF': 109, 'CD': 112, 'CZ': 36, 'CR': 32, 'CU': 72, 'KE': 65, 'KH': 95, 'SV': 53, 'SK': 69, 'KR': 87, 'KW': 92, 'SN': 47, 'SL': 97, 'KZ': 111, 'SA': 56, 'SG': 66, 'SE': 24, 'DO': 11, 'DJ': 115, 'DK': 76, 'DE': 10, 'DZ': 80, 'MK': 105, -99: 1, 'LB': 81, 'TW': 29, 'TR': 70, 'TN': 85, 'LT': 103, 'LU': 59, 'TH': 79, 'TG': 86, 'LY': 108, 'AE': 37, 'VE': 14, 'IS': 107, 'IT': 18, 'AO': 71, 'AR': 13, 'AU': 63, 'AT': 6, 'IN': 31, 'IE': 5, 'QA': 58, 'MZ': 27}
pais_residencia = dict(zip(pais_residencia.values(), pais_residencia.keys()))
data_pais = []
for i in data.pais_residencia:
    data_pais.append(pais_residencia[i])
data.pais_residencia = data_pais

canal_entrada = {'013': 49, 'KHP': 160, 'KHQ': 157, 'KHR': 161, 'KHS': 162, 'KHK': 10, 'KHL': 0, 'KHM': 12, 'KHN': 21, 'KHO': 13, 'KHA': 22, 'KHC': 9, 'KHD': 2, 'KHE': 1, 'KHF': 19, '025': 159, 'KAC': 57, 'KAB': 28, 'KAA': 39, 'KAG': 26, 'KAF': 23, 'KAE': 30, 'KAD': 16, 'KAK': 51, 'KAJ': 41, 'KAI': 35, 'KAH': 31, 'KAO': 94, 'KAN': 110, 'KAM': 107, 'KAL': 74, 'KAS': 70, 'KAR': 32, 'KAQ': 37, 'KAP': 46, 'KAW': 76, 'KAV': 139, 'KAU': 142, 'KAT': 5, 'KAZ': 7, 'KAY': 54, 'KBJ': 133, 'KBH': 90, 'KBN': 122, 'KBO': 64, 'KBL': 88, 'KBM': 135, 'KBB': 131, 'KBF': 102, 'KBG': 17, 'KBD': 109, 'KBE': 119, 'KBZ': 67, 'KBX': 116, 'KBY': 111, 'KBR': 101, 'KBS': 118, 'KBP': 121, 'KBQ': 62, 'KBV': 100, 'KBW': 114, 'KBU': 55, 'KCE': 86, 'KCD': 85, 'KCG': 59, 'KCF': 105, 'KCA': 73, 'KCC': 29, 'KCB': 78, 'KCM': 82, 'KCL': 53, 'KCO': 104, 'KCN': 81, 'KCI': 65, 'KCH': 84, 'KCK': 52, 'KCJ': 156, 'KCU': 115, 'KCT': 112, 'KCV': 106, 'KCQ': 154, 'KCP': 129, 'KCS': 77, 'KCR': 153, 'KCX': 120, 'RED': 8, 'KDL': 158, 'KDM': 130, 'KDN': 151, 'KDO': 60, 'KDH': 14, 'KDI': 150, 'KDD': 113, 'KDE': 47, 'KDF': 127, 'KDG': 126, 'KDA': 63, 'KDB': 117, 'KDC': 75, 'KDX': 69, 'KDY': 61, 'KDZ': 99, 'KDT': 58, 'KDU': 79, 'KDV': 91, 'KDW': 132, 'KDP': 103, 'KDQ': 80, 'KDR': 56, 'KDS': 124, 'K00': 50, 'KEO': 96, 'KEN': 137, 'KEM': 155, 'KEL': 125, 'KEK': 145, 'KEJ': 95, 'KEI': 97, 'KEH': 15, 'KEG': 136, 'KEF': 128, 'KEE': 152, 'KED': 143, 'KEC': 66, 'KEB': 123, 'KEA': 89, 'KEZ': 108, 'KEY': 93, 'KEW': 98, 'KEV': 87, 'KEU': 72, 'KES': 68, 'KEQ': 138, -99: 6, 'KFV': 48, 'KFT': 92, 'KFU': 36, 'KFR': 144, 'KFS': 38, 'KFP': 40, 'KFF': 45, 'KFG': 27, 'KFD': 25, 'KFE': 148, 'KFB': 146, 'KFC': 4, 'KFA': 3, 'KFN': 42, 'KFL': 34, 'KFM': 141, 'KFJ': 33, 'KFK': 20, 'KFH': 140, 'KFI': 134, '007': 71, '004': 83, 'KGU': 149, 'KGW': 147, 'KGV': 43, 'KGY': 44, 'KGX': 24, 'KGC': 18, 'KGN': 11}
canal_entrada = dict(zip(canal_entrada.values(), canal_entrada.keys()))
data_canal = []
for i in data.canal_entrada:
    data_canal.append(canal_entrada[i])
data.canal_entrada = data_canal

indfall = {2: 'no', 1: 'yes', 0: 'unknown'}
data_fall = []
for i in data.indfall:
    data_fall.append(indfall[i])
data.indfall = data_fall

tiprel_1mes = {0:'others', 1:'inactive', 2:'active', 3:'former customer', 4:'Potential'}
data_tip = []
for i in data.tiprel_1mes:
    data_tip.append(tiprel_1mes[i])
data.tiprel_1mes = data_tip

item = ['ahor','aval','cco','cder','cno','ctju','ctma','ctop','ctpp','deco','deme','dela','ecue','fond','hip','plan','pres','reca','tjcr','valo','viv','nomina','pens','recibo'][2:]
data_item = []
for i in data.item:
    data_item.append(item[i-2])
data.item = data_item

ind_actividad_cliente = {0:'inactive customer', 1:'active customer', 2:'unknown'}
data_act = []
for i in data.ind_actividad_cliente:
    data_act.append(ind_actividad_cliente[i])
data.ind_actividad_cliente = data_act

In [76]:
data.pais_residencia.value_counts()

ES     91123
DE        31
FR        27
GB        18
-99       14
IT        14
US        13
VE        12
AR        10
CH        10
BR         8
BE         6
PT         6
NL         6
SG         5
AD         5
MX         4
DO         4
CL         4
PE         4
CO         3
FI         3
GA         3
GQ         3
AT         3
NG         3
MA         2
PA         2
OM         2
AU         2
GR         1
PL         1
Name: pais_residencia, dtype: int64

In [40]:
keepcol = (data.canal_entrada.value_counts()[(data.canal_entrada.value_counts()>1500)]).index
data.canal_entrada = data.canal_entrada.apply(lambda x: 'others' if x not in keepcol else x)

In [41]:
keepcol = (data.pais_residencia.value_counts()[(data.pais_residencia.value_counts()>100)]).index
data.pais_residencia = data.pais_residencia.apply(lambda x: 'others' if x not in keepcol else x)
data.pais_residencia = data.pais_residencia.apply(lambda x: 'others' if x== '-99' else x)

In [42]:
data.user_id = data.user_id.astype(int)

In [43]:
data.to_csv('Santander.csv',index=None)